In [1]:
# only for taurus
import os
print(os.getcwd())
os.chdir("..")
print(os.getcwd())

d:\Uni\KP_MedInf\continuous-aki-predict\model_development\notebooks
d:\Uni\KP_MedInf\continuous-aki-predict\model_development


In [32]:
os.chdir("model_development")


In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
# File paths and separator
DATA_PATH_stages = "data/extracted/kdigo_stages_measured.csv"
DATA_PATH_labs = "data/extracted/labs_original.csv"
DATA_PATH_labs_extended = "data/extracted/labs_extended.csv"
DATA_PATH_labs_new = "data/extracted/labs_new.csv"
# DATA_PATH_vitals = "../data/extracted/vitals-kdigo_stages_measured.csv"
DATA_PATH_vitals = "data/extracted/vitals.csv"
# DATA_PATH_vents = "../data/extracted/vents-vasopressor-sedatives-kdigo_stages_measured.csv"
DATA_PATH_vents = "data/extracted/vents_vasopressor_sedatives.csv"
# DATA_PATH_detail = "../data/extracted/icustay_detail-kdigo_stages_measured.csv"
DATA_PATH_detail = "data/extracted/icustay_detail.csv"
DATA_PATH_heightweight = "data/extracted/heightweight.csv"
DATA_PATH_calcium = "data/extracted/calcium.csv"
DATA_PATH_inr_max = "data/extracted/inr_max.csv"
SEPARATOR = ";"

# Constants
IMPUTE_EACH_ID = False
IMPUTE_COLUMN = False
TESTING = False
TEST_SIZE = 0.05
SPLIT_SIZE = 0.2
MAX_DAYS = 35
CLASS1 = True
ALL_STAGES = False
MAX_FEATURE_SET = True
FIRST_TURN_POS = True
TIME_SAMPLING = True
SAMPLING_INTERVAL = '6H'
RESAMPLE_LIMIT = 16
MOST_COMMON = False
IMPUTE_METHOD = 'most_frequent'
FILL_VALUE = 0
ADULTS_MIN_AGE = 18
ADULTS_MAX_AGE = 120
NORMALIZATION = 'min-max'
HOURS_AHEAD = 48
NORM_TYPE = 'min_max'
RANDOM = 42

def filter_by_length_of_stay(X):
    drop_list = []
    long_stays = X.groupby(['icustay_id']).apply(lambda group: (group['charttime'].max() - group['charttime'].min()).total_seconds() / (24 * 60 * 60) > MAX_DAYS)

    for icustay_id, is_long in long_stays.items():
        if is_long:
            max_time = X[X['icustay_id'] == icustay_id]['charttime'].max() - pd.to_timedelta(MAX_DAYS, unit='D')
            X = X[~((X['icustay_id'] == icustay_id) & (X['charttime'] < max_time))]

    short_stays = X.groupby(['icustay_id']).apply(lambda group: (group['charttime'].max() - group['charttime'].min()).total_seconds() / (24 * 60 * 60) < (HOURS_AHEAD/24))
    drop_list = short_stays[short_stays].index.tolist()

    X = X[~X.icustay_id.isin(drop_list)]
    return X

In [14]:
# Load datasets
print("Loading datasets...")
X = pd.read_csv(DATA_PATH_stages, sep=SEPARATOR)
X.drop(["aki_stage_creat", "aki_stage_uo"], axis=1, inplace=True)
X = X.dropna(how='all', subset=['creat', 'uo_rt_6hr', 'uo_rt_12hr', 'uo_rt_24hr', 'aki_stage'])
X['charttime'] = pd.to_datetime(X['charttime'])

print(len(X))
print(X['aki_stage'].value_counts())

dataset_detail = pd.read_csv(DATA_PATH_detail, sep=SEPARATOR)
# original data
# out data
dataset_detail.drop(['dod', 'admittime', 'dischtime', 'los_hospital', 'ethnicity', 
                     'hospital_expire_flag', 'hospstay_seq', 'first_hosp_stay', 'intime', 
                     'outtime', 'los_icu', 'icustay_seq', 'first_icu_stay', 'ethnicity_grouped'], axis=1, inplace=True)


dataset_labs = pd.read_csv(DATA_PATH_labs, sep=SEPARATOR)
dataset_labs = dataset_labs.dropna(subset=['charttime']).dropna(subset=dataset_labs.columns[4:], how='all')
dataset_labs['charttime'] = pd.to_datetime(dataset_labs['charttime'])
dataset_labs = dataset_labs.sort_values(by=['icustay_id', 'charttime'])
dataset_labs.drop(['albumin_min', 'albumin_max','bilirubin_min', 'bilirubin_max','bands_min', 'bands_max',
                   'lactate_min', 'lactate_max','platelet_min', 'platelet_max','ptt_min', 'ptt_max', 
                   'inr_min', 'inr_max', 'pt_min', 'pt_max'], axis = 1, inplace = True)
# Calculate mean for each pair and drop original columns
column_pairs = [('aniongap_min', 'aniongap_max'), ('albumin_min', 'albumin_max'), 
                ('bands_min', 'bands_max'), ('bicarbonate_min', 'bicarbonate_max'), 
                ('bilirubin_min', 'bilirubin_max'), ('creatinine_min', 'creatinine_max'), 
                ('chloride_min', 'chloride_max'), ('glucose_min', 'glucose_max'), 
                ('hematocrit_min', 'hematocrit_max'), ('hemoglobin_min', 'hemoglobin_max'), 
                ('lactate_min', 'lactate_max'), ('platelet_min', 'platelet_max'), 
                ('potassium_min', 'potassium_max'), ('ptt_min', 'ptt_max'), 
                ('inr_min', 'inr_max'), ('pt_min', 'pt_max'), ('sodium_min', 'sodium_max'), 
                ('bun_min', 'bun_max'), ('wbc_min', 'wbc_max')]

for min_col, max_col in column_pairs:
    try:
        mean_col = min_col.rsplit('_', 1)[0] + '_mean'
        dataset_labs[mean_col] = dataset_labs[[min_col, max_col]].mean(axis=1)
        dataset_labs.drop([min_col, max_col], axis=1, inplace=True)
    except:
        pass

dataset_labs_extended = pd.read_csv(DATA_PATH_labs_extended, sep=SEPARATOR)
dataset_labs_extended = dataset_labs_extended.dropna(subset=['charttime']).dropna(subset=dataset_labs_extended.columns[4:], how='all')
dataset_labs_extended['charttime'] = pd.to_datetime(dataset_labs_extended['charttime'])
dataset_labs_extended = dataset_labs_extended.sort_values(by=['icustay_id', 'charttime'])

column_pairs_extended = [('aniongap_min', 'aniongap_max'), ('albumin_min', 'albumin_max'), 
                ('bands_min', 'bands_max'), ('bicarbonate_min', 'bicarbonate_max'), 
                ('bilirubin_min', 'bilirubin_max'), ('creatinine_min', 'creatinine_max'), 
                ('chloride_min', 'chloride_max'), ('glucose_min', 'glucose_max'), 
                ('hematocrit_min', 'hematocrit_max'), ('hemoglobin_min', 'hemoglobin_max'), 
                ('lactate_min', 'lactate_max'), ('platelet_min', 'platelet_max'), 
                ('potassium_min', 'potassium_max'), ('ptt_min', 'ptt_max'), 
                ('inr_min', 'inr_max'), ('pt_min', 'pt_max'), ('sodium_min', 'sodium_max'), 
                ('bun_min', 'bun_max'), ('wbc_min', 'wbc_max'), 
                ('gfr_min', 'gfr_max'), ('phosphate_min', 'phosphate_max'),('uric_acid_min', 'uric_acid_max'), 
                ('calcium_min', 'calcium_max')]


for min_col, max_col in column_pairs_extended:
    mean_col = min_col.rsplit('_', 1)[0] + '_mean'
    dataset_labs_extended[mean_col] = dataset_labs_extended[[min_col, max_col]].mean(axis=1)
    dataset_labs_extended.drop([min_col, max_col], axis=1, inplace=True)
dataset_labs_extended.drop(['gfr_mean'], axis=1, inplace=True)

dataset_vitals = pd.read_csv(DATA_PATH_vitals, sep=SEPARATOR)
dataset_vents = pd.read_csv(DATA_PATH_vents, sep=SEPARATOR)
dataset_vitals.drop(["heartrate_min", "heartrate_max", "sysbp_min", "sysbp_max", "diasbp_min", "diasbp_max",
                        'meanbp_min', 'meanbp_max', 'tempc_min', 'tempc_max', "resprate_min", "resprate_max", 
                        "spo2_min", "spo2_max", "glucose_min", "glucose_max"], axis=1, inplace=True)
dataset_vitals['charttime'] = pd.to_datetime(dataset_vitals['charttime'])
dataset_vents['charttime'] = pd.to_datetime(dataset_vents['charttime'])
dataset_vitals = dataset_vitals.dropna(subset=dataset_vitals.columns[4:], how='all')
dataset_vitals = dataset_vitals.sort_values(by=['icustay_id', 'charttime'])
dataset_vents = dataset_vents.sort_values(by=['icustay_id', 'charttime'])

dataset_heightweight = pd.read_csv(DATA_PATH_heightweight, sep=SEPARATOR)
dataset_heightweight = dataset_heightweight.dropna(subset=['icustay_id', 'height_first', 'weight_first'], how='all')
dataset_heightweight = dataset_heightweight.sort_values(by=['icustay_id'])

dataset_calcium = pd.read_csv(DATA_PATH_calcium, sep=SEPARATOR)
dataset_calcium.drop(["hadm_id"], axis=1, inplace=True)
dataset_calcium['charttime'] = pd.to_datetime(dataset_calcium['charttime'])
dataset_calcium = dataset_calcium.sort_values(by=['icustay_id', 'charttime'])
dataset_calcium.drop(["subject_id"], axis=1, inplace=True)

dataset_inr_max = pd.read_csv(DATA_PATH_inr_max, sep=SEPARATOR)
dataset_inr_max.drop(["hadm_id", "subject_id"], axis=1, inplace=True)
dataset_inr_max = dataset_inr_max.sort_values(by=['icustay_id'])



# Merge datasets
if MAX_FEATURE_SET:
    
    # Perform merge operations and then drop the 'subject_id' column
    X_extended = X.merge(dataset_labs_extended, on=["icustay_id", "charttime"], how="outer") \
                   .merge(dataset_vitals, on=["icustay_id", "charttime", "subject_id", "hadm_id"], how="outer") \
                   .merge(dataset_vents, on=["icustay_id", "charttime"], how="outer") \
                   .merge(dataset_calcium, on=["icustay_id", "charttime"], how="outer")
    X_extended.drop(["subject_id"], axis=1, inplace=True)
    
    # X_original = X.merge(dataset_labs, on=["icustay_id", "charttime"], how="outer") \
    #               .merge(dataset_vitals, on=["icustay_id", "charttime", "subject_id", "hadm_id"], how="outer") \
    #               .merge(dataset_vents, on=["icustay_id", "charttime"], how="outer") \
    #               .merge(dataset_calcium, on=["icustay_id", "charttime"], how="outer")
    # X_original.drop(["subject_id"], axis=1, inplace=True)

Loading datasets...
3737147
aki_stage
0    3000795
2     353266
3     207759
1     175327
Name: count, dtype: int64


In [13]:
# only use subset of X for testing
unique_ids_subset = X['icustay_id'].unique()[:100]  # Get the first 100 unique icustay_id values
X = X[X['icustay_id'].isin(unique_ids_subset)]  # Filter rows in X
X_extended = X_extended[X_extended['icustay_id'].isin(unique_ids_subset)]
X_original = X_original[X_original['icustay_id'].isin(unique_ids_subset)]

In [13]:
print(len(X.columns))
print(len(X_extended.columns))
print(len(X_original.columns))

42
42
31


In [15]:
print("Filtering patients by age and length of stay...")
# Filtering patients by age and length of stay
dataset_detail = dataset_detail[dataset_detail['admission_age'] >= ADULTS_MIN_AGE]
adults_icustay_id_list = dataset_detail['icustay_id'].unique()
X = X[X.icustay_id.isin(adults_icustay_id_list)].sort_values(by=['icustay_id', 'charttime'])
X_extended = X_extended[X_extended.icustay_id.isin(adults_icustay_id_list)].sort_values(by=['icustay_id', 'charttime'])
# X_original = X_original[X_original.icustay_id.isin(adults_icustay_id_list)].sort_values(by=['icustay_id', 'charttime'])

# X = filter_by_length_of_stay(X)
X_extended = filter_by_length_of_stay(X_extended)
# X_original = filter_by_length_of_stay(X_original)
dataset_detail = dataset_detail[dataset_detail.icustay_id.isin(X['icustay_id'].unique())].sort_values(by=['icustay_id'])

Filtering patients by age and length of stay...


C:\Users\derqu\AppData\Local\Temp\ipykernel_23848\1803750554.py:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  long_stays = X.groupby(['icustay_id']).apply(lambda group: (group['charttime'].max() - group['charttime'].min()).total_seconds() / (24 * 60 * 60) > MAX_DAYS)
C:\Users\derqu\AppData\Local\Temp\ipykernel_23848\1803750554.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  short_stays = X.groupby(

In [18]:
label = ['aki_stage']
skip = ['icustay_id', 'charttime', 'aki_stage']
discrete_feat = ['sedative', 'vasopressor', 'vent', 'hadm_id']
skip.extend(discrete_feat)    
numeric_feat = list(X_extended.columns.difference(skip))

In [19]:
print(len(numeric_feat))

35


In [21]:
# save to csv
X_original.to_csv('data/analysis/data_original_preprocessed_filtered_before_resampling.csv', index=False)
X_extended.to_csv('data/analysis/data_extended_preprocessed_filtered_before_resampling.csv', index=False)


In [4]:
X = pd.read_csv('data/analysis/data_extended_preprocessed_filtered_before_resampling.csv')
# column_test = pd.read_csv('data\preprocessed\preprocessed_data_24H.csv')

In [5]:
print(len(X.columns))

42


In [11]:
column_test.columns

Index(['icustay_id', 'charttime', 'aniongap_mean', 'bicarbonate_mean',
       'bun_mean', 'calcium', 'chloride_mean', 'creat', 'creatinine_mean',
       'diasbp_mean', 'glucose_mean_x', 'glucose_mean_y', 'heartrate_mean',
       'hematocrit_mean', 'hemoglobin_mean', 'meanbp_mean', 'potassium_mean',
       'resprate_mean', 'sodium_mean', 'spo2_mean', 'sysbp_mean', 'tempc_mean',
       'uo_rt_12hr', 'uo_rt_24hr', 'uo_rt_6hr', 'wbc_mean', 'sedative',
       'vasopressor', 'vent', 'hadm_id', 'aki_stage', 'admission_age',
       'gender_F', 'gender_M', 'weight_first', 'height_first', 'inr_max'],
      dtype='object')

In [12]:
column_test.drop(['icustay_id', 'charttime', 'aki_stage', 'hadm_id'], axis=1, inplace=True)

In [13]:
print(len(column_test.columns))

33


In [7]:
# categorical features
dataset_detail = pd.read_csv(DATA_PATH_detail, sep=SEPARATOR)
# original data
# out data
dataset_detail.drop(['dod', 'admittime', 'dischtime', 'los_hospital', 'ethnicity', 
                     'hospital_expire_flag', 'hospstay_seq', 'first_hosp_stay', 'intime', 
                     'outtime', 'los_icu', 'icustay_seq', 'first_icu_stay', 'ethnicity_grouped'], axis=1, inplace=True)
dataset_heightweight = pd.read_csv(DATA_PATH_heightweight, sep=SEPARATOR)
dataset_heightweight = dataset_heightweight.dropna(subset=['icustay_id', 'height_first', 'weight_first'], how='all')
dataset_heightweight = dataset_heightweight.sort_values(by=['icustay_id'])
dataset_inr_max = pd.read_csv(DATA_PATH_inr_max, sep=SEPARATOR)
dataset_inr_max.drop(["hadm_id", "subject_id"], axis=1, inplace=True)
dataset_inr_max = dataset_inr_max.sort_values(by=['icustay_id'])

In [58]:
# Merging not time-dependent data
dataset_detail_merging = dataset_detail[dataset_detail['icustay_id'].isin(X['icustay_id'].unique())].sort_values(by=['icustay_id'])
# theirs
# dataset_detail_merging = pd.get_dummies(dataset_detail_merging, columns=['gender', 'ethnicity_grouped'])
#ours
dataset_detail_merging = pd.get_dummies(dataset_detail_merging, columns=['gender'])
# dataset_detail_merging.drop(['subject_id', 'hadm_id'], axis=1, inplace=True)

X = X.merge(dataset_detail_merging, on='icustay_id')
# X = X.merge(dataset_heightweight, on='icustay_id')
# X = X.merge(dataset_inr_max, on='icustay_id')

In [59]:
X.drop(['subject_id', 'hadm_id_x', 'hadm_id_y', 'aki_stage'], axis=1, inplace=True)

In [60]:
print(column_test.columns)

Index(['icustay_id', 'charttime', 'aniongap_mean', 'bicarbonate_mean',
       'bun_mean', 'calcium', 'chloride_mean', 'creat', 'creatinine_mean',
       'diasbp_mean', 'glucose_mean_x', 'glucose_mean_y', 'heartrate_mean',
       'hematocrit_mean', 'hemoglobin_mean', 'meanbp_mean', 'potassium_mean',
       'resprate_mean', 'sodium_mean', 'spo2_mean', 'sysbp_mean', 'tempc_mean',
       'uo_rt_12hr', 'uo_rt_24hr', 'uo_rt_6hr', 'wbc_mean', 'sedative',
       'vasopressor', 'vent', 'aki_stage', 'admission_age', 'gender_F',
       'gender_M'],
      dtype='object')


In [61]:
print(len(X.columns))

32


In [65]:
print(X.columns)

Index(['icustay_id', 'charttime', 'creat', 'uo_rt_6hr', 'uo_rt_12hr',
       'uo_rt_24hr', 'aniongap_mean', 'bicarbonate_mean', 'creatinine_mean',
       'chloride_mean', 'glucose_mean_x', 'hematocrit_mean', 'hemoglobin_mean',
       'potassium_mean', 'sodium_mean', 'bun_mean', 'wbc_mean',
       'heartrate_mean', 'sysbp_mean', 'diasbp_mean', 'meanbp_mean',
       'resprate_mean', 'tempc_mean', 'spo2_mean', 'glucose_mean_y', 'vent',
       'vasopressor', 'sedative', 'calcium', 'admission_age', 'gender_F',
       'gender_M'],
      dtype='object')


In [27]:
# print difference between X.columns and column_test.columns
print(set(X.columns) - set(column_test.columns))

set()


In [66]:
# group by icustay_id and save all rows of 10 random ids ommiting the icustay_id
X_grouped = X.groupby('icustay_id')
# pick 10 random ids
ids = np.random.choice(X['icustay_id'].unique(), 10)
os.makedirs('data/dummy_data_lab_values', exist_ok=True)
for id in ids:
    # remove icustay_id from the group
    group = X_grouped.get_group(id).drop('icustay_id', axis=1)
    # save to csv
    group.to_csv(f'data/dummy_data_lab_values/{id}_original_preprocessed_filtered_before_resampling.csv', index=False)

### create unresampled shifted dataset

In [18]:
X = X_extended.copy()

In [49]:
print(X['aki_stage'].isna().sum())


11478


#### Option1: fill all labels since past

In [9]:
# binarize aki_stage labels
X['aki_stage'] = (X['aki_stage'] > 0).astype(int)
# create X_shifted where, for each icuststay_id, an aki_stage > 0 is also applied to all timestamps in the 48 hours before
import pandas as pd
import numpy as np

# Assuming X is your original dataframe
# First, make sure X is sorted by icustay_id and charttime
X = X.sort_values(['icustay_id', 'charttime'])

# Create a copy of X to modify
X_shifted = X.copy()

# Convert charttime to datetime if it's not already
X_shifted['charttime'] = pd.to_datetime(X_shifted['charttime'])

# Group by icustay_id
grouped = X_shifted.groupby('icustay_id')

# Function to shift aki_stage
def shift_aki_stage(group):
    # Find where aki_stage > 0
    aki_events = group[group['aki_stage'] > 0]
    
    for _, event in aki_events.iterrows():
        # Calculate the time 48 hours before the event
        time_before = event['charttime'] - pd.Timedelta(hours=48)
        
        # Set aki_stage to 1 for all rows in the 48-hour window before the event
        mask = (group['charttime'] >= time_before) & (group['charttime'] <= event['charttime'])
        group.loc[mask, 'aki_stage'] = 1
    
    return group

# Apply the function to each group
X_shifted = grouped.apply(shift_aki_stage)

# Reset index if needed
X_shifted = X_shifted.reset_index(drop=True)

/tmp/ipykernel_1312113/378773068.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  X_shifted = grouped.apply(shift_aki_stage)


In [12]:
# print number of rows in X where aki_stage is nan
print(X_shifted['aki_stage'].isna().sum())
print(X['aki_stage'].isna().sum())


0
7536107


In [13]:
print(X['aki_stage'].value_counts(normalize=True) * 100)
print(X_shifted['aki_stage'].value_counts(normalize=True) * 100)


aki_stage
0.0    79.691991
2.0     9.808070
3.0     5.596306
1.0     4.903633
Name: proportion, dtype: float64
aki_stage
1    51.789364
0    48.210636
Name: proportion, dtype: float64


In [14]:
# save to csv
X_shifted.to_csv('data/analysis/data_shifted.csv', index=False)

#### Option2: fill only window in the past

In [16]:
import pandas as pd
import numpy as np

# Assuming X is your original dataframe
# First, make sure X is sorted by icustay_id and charttime
X = X.sort_values(['icustay_id', 'charttime'])

# Create a copy of X to modify
X_shifted_window = X.copy()

# Convert charttime to datetime if it's not already
X_shifted_window['charttime'] = pd.to_datetime(X_shifted_window['charttime'])

# Group by icustay_id
grouped = X_shifted_window.groupby('icustay_id')

# Function to shift aki_stage
def shift_aki_stage(group):
    # Find where aki_stage > 0
    aki_events = group[group['aki_stage'] > 0].copy()
    
    for idx, event in aki_events.iterrows():
        # Calculate the time 48 and 40 hours before the event
        time_before_48 = event['charttime'] - pd.Timedelta(hours=48)
        time_before_40 = event['charttime'] - pd.Timedelta(hours=40)
        
        # Find rows in the 40-48 hour window before the event
        mask = (group['charttime'] >= time_before_48) & (group['charttime'] <= time_before_40)
        rows_to_update = group[mask]
        
        if not rows_to_update.empty:
            # Update the aki_stage for these rows
            group.loc[rows_to_update.index, 'aki_stage'] = 1
            
            # Remove the original event
            group.loc[idx, 'aki_stage'] = 0
    
    return group

# Apply the function to each group
X_shifted_window = grouped.apply(shift_aki_stage)

# Reset index if needed
X_shifted_window = X_shifted_window.reset_index(drop=True)

/tmp/ipykernel_1312113/181169435.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  X_shifted_window = grouped.apply(shift_aki_stage)


In [19]:
# print number of rows in X where aki_stage is nan
print(X_shifted_window['aki_stage'].isna().sum())
print(X['aki_stage'].isna().sum())


5970197
7536107


In [20]:
print(X['aki_stage'].value_counts(normalize=True) * 100)
print(X_shifted_window['aki_stage'].value_counts(normalize=True) * 100)


aki_stage
0.0    79.691991
2.0     9.808070
3.0     5.596306
1.0     4.903633
Name: proportion, dtype: float64
aki_stage
0.0    51.128208
1.0    47.391171
2.0     1.198435
3.0     0.282187
Name: proportion, dtype: float64


In [22]:
X_shifted_window.to_csv('data/analysis/data_shifted_window.csv', index=False)

## looped dataset creation

In [46]:
import gc
del X
# del X_original
gc.collect()

1714

In [23]:
X = X_extended.copy()

In [35]:
# only take the first 10000 rows of X for test
X = X[:10000]

In [24]:
print('X Columns', len(X.columns))

X Columns 42


In [9]:
SAMPLING_INTERVALS = ['1H', '2H', '4H', '6H', '8H', '12H', '24H']


In [10]:
print(len(X_numeric.columns))

NameError: name 'X_numeric' is not defined

In [25]:
# saving every sampling interval
original = False
# Resampling
for SAMPLING_INTERVAL in SAMPLING_INTERVALS:
    if TIME_SAMPLING:
        
        # Set index and group by 'icustay_id' before resampling
        X_resampled = X.copy()
        X_resampled.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)
        
        # Resample and aggregate features
        if MAX_FEATURE_SET:
            X_discrete = X_resampled[discrete_feat].max().fillna(FILL_VALUE).astype(np.int64)
        X_numeric = X_resampled[numeric_feat].mean()
        X_label = X_resampled['aki_stage'].max()

        print("Merging sampled features")
        try:
            X_resampled = pd.concat([X_numeric, X_discrete, X_label], axis=1).reset_index()
        except:
            X_resampled = pd.concat([X_numeric, X_label], axis=1).reset_index()

    # Forward fill again after resampling
    X_resampled['aki_stage'] = X_resampled.groupby('icustay_id')['aki_stage'].ffill(limit=RESAMPLE_LIMIT).fillna(0)
    print('X Columns', len(X_resampled.columns))

    # Ensure binary values (convert any positive number to 1)
    X_resampled['aki_stage'] = (X_resampled['aki_stage'] > 0).astype(int)

    # Shifting labels
    shift_steps = HOURS_AHEAD // int(SAMPLING_INTERVAL[:-1])
    X_resampled['aki_stage'] = X_resampled.groupby('icustay_id')['aki_stage'].shift(-shift_steps)
    X_resampled = X_resampled.dropna(subset=['aki_stage'])
    print('X Columns', len(X_resampled.columns))
    
    # Merging not time-dependent data
    dataset_detail_merging = dataset_detail[dataset_detail['icustay_id'].isin(X['icustay_id'].unique())].sort_values(by=['icustay_id'])
    if original:
        # theirs
        dataset_detail_merging = pd.get_dummies(dataset_detail_merging, columns=['gender', 'ethnicity_grouped'])
    else:
        dataset_detail_merging = pd.get_dummies(dataset_detail_merging, columns=['gender'])
        X_resampled = X_resampled.merge(dataset_heightweight, on='icustay_id')
        X_resampled = X_resampled.merge(dataset_inr_max, on='icustay_id')

    X_resampled = X_resampled.merge(dataset_detail_merging, on='icustay_id')
    print('X Columns', len(X_resampled.columns))


    # If no imputation method selected or only impute each id, for the remaining nan impute direclty with FILL_VALUE
    X_resampled = X_resampled.fillna(FILL_VALUE) 


    # Save preprocessed data
    X_resampled.to_csv(f'data/preprocessed/preprocessed_data_extended_{SAMPLING_INTERVAL}.csv', index=False)
    print('X Columns', len(X_resampled.columns))




C:\Users\derqu\AppData\Local\Temp\ipykernel_23848\1200676958.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  X_resampled.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)


MemoryError: Unable to allocate 3.25 GiB for an array with shape (40, 10893536) and data type float64

In [27]:
X = X_original

## normal ds creation

In [28]:
# Resampling
if TIME_SAMPLING:
    
    label = ['aki_stage']
    skip = ['icustay_id', 'charttime', 'aki_stage']
    discrete_feat = ['sedative', 'vasopressor', 'vent', 'hadm_id']
    skip.extend(discrete_feat)    
    numeric_feat = list(X.columns.difference(skip))
    
    # Set index and group by 'icustay_id' before resampling
    X = X.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)
    
    # Resample and aggregate features
    if MAX_FEATURE_SET:
        X_discrete = X[discrete_feat].max().fillna(FILL_VALUE).astype(np.int64)
    X_numeric = X[numeric_feat].mean()
    X_label = X['aki_stage'].max()

    print("Merging sampled features")
    try:
        X = pd.concat([X_numeric, X_discrete, X_label], axis=1).reset_index()
    except:
        X = pd.concat([X_numeric, X_label], axis=1).reset_index()



# Forward fill again after resampling
X['aki_stage'] = X.groupby('icustay_id')['aki_stage'].ffill(limit=RESAMPLE_LIMIT).fillna(0)



# Ensure binary values (convert any positive number to 1)
X['aki_stage'] = (X['aki_stage'] > 0).astype(int)

# Shifting labels
shift_steps = HOURS_AHEAD // int(SAMPLING_INTERVAL[:-1])
X['aki_stage'] = X.groupby('icustay_id')['aki_stage'].shift(-shift_steps)
X = X.dropna(subset=['aki_stage'])

# Merging not time-dependent data
dataset_detail_merging = dataset_detail[dataset_detail['icustay_id'].isin(X['icustay_id'].unique())].sort_values(by=['icustay_id'])
# theirs
# dataset_detail = pd.get_dummies(dataset_detail, columns=['gender', 'ethnicity_grouped'])
#ours
dataset_detail_merging = pd.get_dummies(dataset_detail_merging, columns=['gender'])
dataset_detail_merging.drop(['subject_id', 'hadm_id'], axis=1, inplace=True)
X = X.merge(dataset_detail_merging, on='icustay_id')
X = X.merge(dataset_heightweight, on='icustay_id')
X = X.merge(dataset_inr_max, on='icustay_id')

# If no imputation method selected or only impute each id, for the remaining nan impute direclty with FILL_VALUE
X = X.fillna(FILL_VALUE) 


/tmp/ipykernel_924951/2522398281.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  X = X.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)


Merging sampled features


In [29]:

# Save preprocessed data
X.to_csv('data/preprocessed/preprocessed_data_original.csv', index=False)